# MC_sample_geneartion:

This skript draws random demand samples and solves the state equations. The results are saved to be used in
the MC-SIR algorithm and to train the DNN.

If the folder '/data_files/results_***testcase*** does not yet exist for the ***testcase*** of interest pls add this first.

### Imports:

In [6]:
import tensorflow as tf
import sys
from libdnnmcmc.utility import load_scenario
from libdnnmcmc.steady_state_solvers import SE_solver

## Settings:
- testcase: testcase for which samples are to be generated
- n_samples: number of samples
- precision: the samples are solved up to $\Psi \leq precision$
- name_ext: the save file's name is extended by name_ext; <br>
name_ext is expected to be \[1, 2, 3, ...\] if not all MC samples are saved in one file

In [8]:
testcase = 'loop'
n_samples = 30
precision = 1.e-5
file_ext = 'test'       # file_ext = sys.argv[1]

In [ ]:
# load testcae:

SE, VM, d_prior_dist, data_file = load_scenario(testcase)
cycles = VM.cycles
_solve_SE = SE_solver(nr_prec=precision)

In the following we define a small wrapper around the solver-instance (*\_solve\_SE*) to handle the data assignments.
SEt verbose = True to print information regarding the solver's internal state.

In [9]:
def solve_SE(demand, verbose=False):
    SE.load_save_state()
    for dem in SE.demands.keys():
        ind = SE.dem_ind[dem]
        d = demand[ind]
        SE.Q_heat[ind].assign(d)
    _solve_SE(SE, cycles, verbose=verbose)
    return tf.concat([SE.T, SE.mf, SE.p, SE.T_end], axis=0)

Sample demands and save corresponding grid states:

In [11]:
demands = d_prior_dist.sample(n_samples)
for i, demand in enumerate(demands):
    if i % 10 == 0:
        print(f'calculate sample {i}')
    state = solve_SE(demand)
    tf.print(demand, tf.transpose(state), summarize=-1, output_stream='file://' + f'{data_file}{file_ext}.csv')

calculate sample 0
calculate sample 10
calculate sample 20
